<a href="https://colab.research.google.com/github/ngzhiwei517/Machine_Learning_Self-Learn/blob/main/Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Data Mining

# 1.1 Lake dataset

After surfing through the Internet, we found that we were unable to find any historical data about the water level of the dam in Malaysia. The most we can get is the current data which is too little to train our model even if we start collecting it by April 2024. Hence, we decided to look for data out of Malaysia.

Luckily for us, we were able to find a website which contain the historical data regarding a few lakes located in Phoenix, AZ, USA. In order to extract those data out and store it in a .csv file, we have to perform Web Scrapping.



#1.2 Inspecting the website (Lake dataset)

Before scrapping the data out from the website, we make some discovering about the website first and understand where the data is stored and how to extract the data for a period of time by performing the following code.

We need the help of some packages in order to perform web scrapping.

* BeautifulSoup
* requests

In [ ]:
# Import required packages
from bs4 import BeautifulSoup
import requests

First, we try to get request from the website.

Source: https://www.watershedconnection.com/

In [ ]:
# Website for data mining
url = "https://streamflow.watershedconnection.com/DWR?reportDate=2017-1-1"

In [ ]:
# Get requests from website
requests.get(url)

It returns us a response 200. Response 200 indicates that our request to the website has succeeded.

Now, we try to get the HTML of the website.

In [ ]:
# Get requests from website
page_lake_test = requests.get(url)

In [ ]:
# Get the html of the website
soup_lake_test = BeautifulSoup(page_lake_test.text, 'html')

In [ ]:
# Print the html of the website
print(soup_lake_test.prettify())

Well, there is quite a lot of code in the HTML. By inspecting the website, we know that the table containing the data is code under the tags 'table'. Let's have a search for the tags 'table'.

In [ ]:
# Find all 'table' in the html (Searching for the tags containing the table of the data)
soup_lake_test.find_all('td')

There is the tags 'table' containing the data we want. However, there are also other tables in that website too. We select the table we want by adding a '[0]' at the end of the previous code.

In [ ]:
# Find all 'table' in the html (Searching for the tags containing the table of the data)
soup_lake_test.find_all('table')[0]

The table we wanted has been selected. From the code, we able to see that the data we wanted is under the tags 'td'. Let's try to get the data out.

In [ ]:
# Find all 'td' in the html (Searching for the data we wanted)
soup_lake_test.find_all('td')

Let's pull the first data out from the tags to see whether the data is in the format that we wanted.

In [ ]:
# Pulling text information from the website
soup_lake_test.find('td').text

Well, almost there but it containing something that we don't want. Let's trim it off.

In [ ]:
# Pulling text information from the website
soup_lake_test.find('td').text.strip()

Nice, this is what we wanted.

The websites also containing some information of the weather for that respective day. From the inspection of the websites, the weather data is code under the 'b' tags.

In [ ]:
# Find all 'b' in the html (Searching for the weather data)
soup_lake_test.find_all('b')

Each data is in the format highest/lowest. So, let us try to separate the first data.

In [ ]:
# Separating the data and removing unwanted value from the data
temperature_test = soup_lake_test.find_all('b')[0]
temperature_list_test = temperature_test.text.strip().split("°")
temperature_list_test[1] = temperature_list_test[1].replace("/", "")
temperature_list_test

From the above list, we know that the data one is the first and the second items in the list.

Since we know where our data is located in the website, it's time to extract those data and store it in a dataframe.

In [ ]:
# Selecting the table needed
table_lake_test = soup_lake_test.find_all('table')[0]

Before creating the dataframe, we need to set the feature names first. The feature names are under the tags 'th'. Let's have a look of it.

In [ ]:
# Find all 'th' in the html (Searching for the feature names)
soup_lake_test.find_all('th')

That was an unexpected coding. We have no choice to set the feature names ourself.

In [ ]:
# Setting the feature names manually into a list
feature_names_lake_test = ['Location', 'Full (%)', 'Current Elevation(ft)',
                      'Current Storage (af)', 'Remaining Elevation (ft)',
                      'Available Storage (af)', '24 hr. Change',
                      'Rain (inches)']
feature_names_lake_test

This is better. Now we will create a dataframe and set the feature names into it.

In [ ]:
import pandas as pd

In [ ]:
lake_test_df = pd.DataFrame(columns = feature_names_lake_test)
lake_test_df

Now it is time to abtract the data we wanted and storing it into the dataframe.

In [ ]:
column_data_lake_test = table_lake_test.find_all('tr')
for row in column_data_lake_test[2:]:
  row_data = row.find_all('td')
  individual_row_data = [data.text.strip() for data in row_data]

  length = len(lake_test_df)
  lake_test_df.loc[length] = individual_row_data

lake_test_df

Seems like the web scrapping works for this website as the data we wanted is successfully extract from the website into the dataframe. Now, we will be moving to the next step which is extracting data for a range of time.



# 1.3 Extracting data for a range of time (Lake dataset)

Setting up the date range for the data we wanted to extract.

In [ ]:
# Set Date range
date_start_text = "2013-1-1"
date_end_text = "2023-1-1"
freq_text = "1"

The range of our data gathering will be daily for a period of 10 years.

The convertion from text to date is necessary to be put as the range of the while loop.

In [ ]:
# Convert text to date
date_start_inc = date_start_text
date_start_inc_date = datetime.strptime(date_start_inc, "%Y-%m-%d")
date_end_date = datetime.strptime(date_end_text, "%Y-%m-%d")
freq_obj = int(freq_text)

date_start_inc_date = date_start_inc_date.date()
date_end_date = date_end_date.date()

In [ ]:
# Setting up features name
feature_names_lake = [ 'Location', 'Full_%', 'Current_Elevation_ft',
                      'Current_Storage_af', 'Remaining_Elevation_ft',
                      'Available_Storage_af', '24hr.Change',
                      'Rain_inches', 'Date', 'Highest_temperature',
                       'Lowest_temperature','Highest_humidity',
                       'Lowest_humidity']
feature_names_lake

In [ ]:
# Setting up the dataframe to store data later with the feature names we set previously
lake_df = pd.DataFrame(columns = feature_names_lake)

Our dataframe is ready to store data gather from the website

In [ ]:
# Using the while loop to gather data for a period of time
while date_start_inc_date <= date_end_date:
  # Date will be automatically insert to the back of the url to access data for that particular day
  url = 'https://streamflow.watershedconnection.com/DWR?reportDate='+ date_start_inc_date.strftime("%Y-%m-%d")
  page_lake = requests.get(url)
  soup_lake = BeautifulSoup(page_lake.text, 'html')
  table_lake = soup_lake.find_all('table')[0]
  column_data_lake = table_lake.find_all('tr')
  for row_lake in column_data_lake[2:]:
    row_data_lake = row_lake.find_all('td')
    individual_row_data_lake = [data.text.strip() for data in row_data_lake]
    individual_row_data_lake.append(date_start_inc_date.strftime("%Y-%m-%d"))
    # Triming the temperature value and adding it into the list
    temperature = soup_lake.find_all('b')[0]
    temperature_list = temperature.text.strip().split("°")
    temperature_list[1] = temperature_list[1].replace("/", "")
    individual_row_data_lake.append(temperature_list[0])
    individual_row_data_lake.append(temperature_list[1])
    # Triming the humidity value and adding it into the list
    humidity = soup_lake.find_all('b')[2]
    humidity_list = humidity.text.strip().split()
    individual_row_data_lake.append(humidity_list[0])
    individual_row_data_lake.append(humidity_list[2])
    # Storing the list of data into the dataframe
    length = len(lake_df)
    lake_df.loc[length] = individual_row_data_lake

  date_start_inc_date = date_start_inc_date + timedelta(days = freq_obj)

From the table above, the data we gather has correctly store in their respective column. A total of 36530 rows of data gather for 10 years period

In [ ]:
lake_df.to_csv(r'/content/sample_data/lake_dataset_10years.csv', index = False)

We save the data gather into a csv file.

After inspecting the dataset, we found out the weather information gather from that website is not precise enough. Hence, we decided to gather the information of the weather from the nearest weather station to the area where the lakes are located from other website.

# 2.1 Weather dataset

First we need to see which weather station is the nearest to that area. Only the weather station at the airport in that area is storing the historical weather data.

The airport available there is as follow:

* Phoenix Sky Harbor International Airport
* Phoenix-Mesa Gateway Airport
* Falcon Field Airport
* Scottsdale Airport
* Phoenix Deer Valley Airport
* Glendale Municipal Airport
* Phoenix-Goodyear Airport

We have chosen Falcon Field Airport since it is nearest to that area.

We now gather the data about the weather information of that area from this website.


---


Source: https://www.visualcrossing.com/

#2.2 Inspecting the website (Weather dataset)

After taking a tour through this website, it's provide API that allow user to get the data from their website easily. However, for free user, a total of 1000 records are allowed for a day. This mean that we need a few day to gather all the data we wanted.

Since API service is provided, it save up a lot of our works to inspect the website.



#2.3 Extracting data for a range of time (Weather dataset)

As same as previous, we first need to indicate the range of date of the data we wanted to extract. Since we are only able to extract limited data per day, in this notebook, we will only be extracting data for a period of one year.

In [ ]:
# Set Date range
date_start_text = "2013-1-1"
date_end_text = "2014-1-1"
freq_text = "1"

In [ ]:
# Convert text to date
date_start_inc = date_start_text
date_start_inc_date = datetime.strptime(date_start_inc, "%Y-%m-%d")
date_end_date = datetime.strptime(date_end_text, "%Y-%m-%d")
freq_obj = int(freq_text)

date_start_inc_date = date_start_inc_date.date()
date_end_date = date_end_date.date()

We now start to extract the data we wanted using the API. We will be getting weather information for a day for each query we make. Hence, we need to create a list to store all the dataframe we get so that we can combine it into one later.

In [ ]:
# Creating the list to store the .csv file
list_of_csv = []
# Creating the while loop
while date_start_inc_date <= date_end_date:
  url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/%20Falcon%20Field%20Airport%2C%204800%20E%20Falcon%20Dr%2C%20Mesa/'+date_start_inc_date.strftime("%Y-%m-%d")+'/' +date_start_inc_date.strftime("%Y-%m-%d")+'?unitGroup=metric&include=days&key={API KEY}&contentType=csv'
  df = pd.read_csv(url, index_col=None, header = 0)
  date_start_inc_date = date_start_inc_date + timedelta(days = freq_obj)
  # Adding the dataframe into the list
  list_of_csv.append(df)

In [ ]:
# Combining all the dataframe in the list into one
weather_2013_df = pd.concat(list_of_csv, axis= 0,ignore_index=True)

In [ ]:
# Saving the dataframe into a .csv file
weather_2013_df.to_csv(r'/content/sample_data/weather_dataset_2013.csv', index = False)

Now, we have the weather dataset for the year 2013. The dataset for the remaining years will be done using another notebook by using the same code as above

Since all of the dataset we wanted are ready, it's time to proceed to the next section,which is Data Preprocessing.